In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0371, -0.0706, -0.0758, -0.1946, -0.0120,  0.1459, -0.1267, -0.2044,
          0.0088,  0.3439],
        [ 0.1763,  0.0303,  0.0126, -0.0795, -0.0372,  0.2128, -0.0321, -0.1466,
          0.0303,  0.2438]], grad_fn=<AddmmBackward>)

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    # MLP是nn.Module的一个子类
    def __init__(self):
        # 调用`MLP`的父类`Block`的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数`params`（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256) #隐藏层
        self.out = nn.Linear(256, 10)    #输出层
        
    # 定义模型的正向传播，即如何根据输入`X`返回所需的模型输出
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.2628, -0.0226,  0.2706, -0.0730, -0.2767, -0.3097,  0.0392, -0.1104,
          0.1161,  0.0077],
        [ 0.3705, -0.0844,  0.1381, -0.0224, -0.2206, -0.2180, -0.0546, -0.1713,
          0.1511, -0.1484]], grad_fn=<AddmmBackward>)

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
            '''这里，`block`是`Module`子类的一个实例。我们把它保存在'Module'类的
             成员变量`_modules` 中。`block`的类型是OrderedDict。'''
    
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0537, -0.1703, -0.0909, -0.1057, -0.0936,  0.1799, -0.0343,  0.1027,
         -0.1369,  0.0365],
        [-0.0208, -0.0456, -0.0144,  0.0246, -0.1173,  0.1345, -0.0331,  0.0833,
         -0.0263,  0.0534]], grad_fn=<AddmmBackward>)

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变。
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及`relu`和`dot`函数。
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数。
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(0.1857, grad_fn=<SumBackward0>)

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1027, grad_fn=<SumBackward0>)